In [2]:
import requests
from requests.auth import HTTPBasicAuth
import json
import pandas as pd
import keyring
import datetime
import os

In [3]:
jira_domain = 'https://forgeholidays.atlassian.net/'
email = 'deepankar.srigyan@sykescottages.co.uk'
project_key = 'DAB ORDER BY created DESC'

#Credential needs to be stored in window credential before running this.
api_token = keyring.get_password('JIRA_API', 'deepankar.srigyan@sykescottages.co.uk')


In [4]:
import datetime
from datetime import date

In [5]:
today = date.today()
date_prefix = today.strftime("%Y%m%d")  # Format the date as YYYYMMDD


#### Working without loop

#### Doesn't loop

In [20]:
# path correction
if not jira_domain.startswith('https://'):
    jira_domain = 'https://' + jira_domain

# Create Jira API URL
url = f"{jira_domain}/rest/api/3/search"

# Query 
query = {
    'jql': f'project={project_key}',
    'maxResults': 10000  # Adjust based on your needs
}

try:
    # request
    response = requests.get(
        url,
        headers={'Content-Type': 'application/json'},
        params=query,
        auth=HTTPBasicAuth(email, api_token),
        verify=True  # SSL security off, need to work with IT so we do not need to do it False
    )

    # request URL (for debugging)
    print("Requested URL:", response.url)

    # errors
    response.raise_for_status()

    # Process the response
    issues = response.json()['issues']
    print(f'Total issues fetched: {len(issues)}')

    # Convert to DataFrame
    df_issues = pd.json_normalize(issues)  # JSON data into a  table

    
    # Construct the file name with the date prefix
    path = r"C:\Users\DeepankarSrigyan\OneDrive - Sykes Cottages Ltd\Documents - Data & Analytics\Insight team\1. Admin\4. Internal reporting"
    # file_name = f"{date_prefix}_jira_issues.xlsx"
    file_name = os.path.join(path, f"{date_prefix}_jira_issues.csv")

    # Save the DataFrame to an Excel file
    df_issues.to_csv(file_name, index=False)

    # Save to an Excel file
    # df_issues.to_excel('jira_issues.xlsx', index=False)  

    print('Issues have been saved to jira_issues.json and jira_issues.xlsx')  # Update print statement

except requests.exceptions.RequestException as e:
    print(f'An error occurred: {e}')

Requested URL: https://forgeholidays.atlassian.net//rest/api/3/search?jql=project%3DDAB+ORDER+BY+created+DESC&maxResults=10000
Total issues fetched: 100
Issues have been saved to jira_issues.json and jira_issues.xlsx


#### Take the data in loop, pagination

In [6]:
import requests
from requests.auth import HTTPBasicAuth
import os
import pandas as pd

# Ensure the JIRA domain starts with 'https://'
if not jira_domain.startswith('https://'):
    jira_domain = 'https://' + jira_domain

# Create JIRA API URL
url = f"{jira_domain}/rest/api/3/search"

# Initialize variables for pagination
start_at = 0
max_results = 100  # Set to the maximum typically allowed by JIRA
total_issues_fetched = 0
all_issues = []

# Loop to fetch issues in pages
while True:
    query = {
        'jql': f'project={project_key}',
        'startAt': start_at * max_results,  # Adjusted for pagination
        'maxResults': max_results
    }

    try:
        response = requests.get(
            url,
            headers={'Content-Type': 'application/json'},
            params=query,
            auth=HTTPBasicAuth(email, api_token),
            verify=True  # Consider setting this to True once SSL certificates are configured
        )
        response.raise_for_status()
        response_data = response.json()
        issues = response_data['issues']
        all_issues.extend(issues)
        total_issues_fetched += len(issues)
        print(f'Fetched {len(issues)} issues, total fetched: {total_issues_fetched}')
        if len(issues) < max_results:
            break  # Exit the loop if the last page of issues is fetched
        else:
            start_at += 1  # Move to the next page
    except requests.exceptions.RequestException as e:
        print(f'An error occurred: {e}')
        break

# Convert to DataFrame
df_issues = pd.json_normalize(all_issues)

# Construct the file name with the date prefix
path = r"C:\Users\DeepankarSrigyan\OneDrive - Sykes Cottages Ltd\Documents - Data & Analytics\Insight team\1. Admin\4. Internal reporting\1. Jira"
file_name = os.path.join(path, f"{date_prefix}_jira_issues.csv")

# Save the DataFrame to a CSV file
df_issues.to_csv(file_name, index=False)

print(f'Total issues fetched: {total_issues_fetched}')
print(f'Issues have been saved to {file_name}')

An error occurred: 400 Client Error: Bad Request for url: https://forgeholidays.atlassian.net//rest/api/3/search?jql=project%3DDAB+ORDER+BY+created+DESC&startAt=0&maxResults=100
Total issues fetched: 0
Issues have been saved to C:\Users\DeepankarSrigyan\OneDrive - Sykes Cottages Ltd\Documents - Data & Analytics\Insight team\1. Admin\4. Internal reporting\1. Jira\20240701_jira_issues.csv
